In [185]:
# https://becominghuman.ai/nlp-classifying-positive-and-negative-restaurant-reviews-bag-of-words-model-31e9abfd7286
# https://www.kaggle.com/omkarsabnis/sentiment-analysis-on-the-yelp-reviews-dataset
import pandas as pd
pd.set_option('max_colwidth', 20)
import re
import matplotlib.pyplot as plt

import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /Users/makhy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [184]:
df = pd.read_csv("Openrice_all_lunch.csv", 
                index_col = 0)

### Select only Rated Reviews

In [6]:
# Among the 2600+ reviews, 634 are without a rating
df.isna().sum()

District              0
Cuisine               0
Reviews               0
Restaurant_Name      58
Rating              634
Cost                  0
Wait_Time           602
Restuaruant_Link      0
dtype: int64

In [59]:
# Retain only rated reviews
df_c = df.dropna(subset = ["Rating"])

In [186]:
# df_c.describe()

In [188]:
df_c.reset_index(drop=True, inplace=True)
# df_c.head()

In [62]:
# df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)
df_c["Rating"] = df_c["Rating"].apply(pd.to_numeric)

/Users/makhy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Classify Ratings into Positive/Nagative Sentiment

In [155]:
def transform_Good_Bad(r):
    if r >= 3:
        return("Good")
    else:
        return("Bad")

In [156]:
add_gnb = lambda x: transform_Good_Bad(x)

In [160]:
df_c["Sentiment"] = df_c["Rating"].apply(add_gnb)

/Users/makhy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [161]:
df_c["Sentiment_Code"] = df_c["Sentiment"].astype("category").cat.codes

/Users/makhy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [182]:
df_c.head()

,District,Cuisine,Reviews,Restaurant_Name,Rating,Cost,Wait_Time,Restuaruant_Link,Grade,Grade_Code,Sentiment,Sentiment_Code
0,Admiralty,American,"I am a burger lover.Today,...",Shake Shack,4.8,140,0.0,www.openrice.com/en/hongko...,Excellent,0,Good,1
1,Admiralty,American,Its a very famouly burger ...,Shake Shack,4.5,100,20.0,www.openrice.com/en/hongko...,Excellent,0,Good,1
2,Admiralty,American,Love the ambiance and comf...,Commissary,3.7,170,0.0,www.openrice.com/en/hongko...,Good,1,Good,1
3,Admiralty,American,I'm an old fashioned perso...,Ruth's Chris Steak House,3.8,350,NaN,www.openrice.com/en/hongko...,Good,1,Good,1
4,Admiralty,American,"Our China colleague came, ...",Commissary,3.2,180,0.0,www.openrice.com/en/hongko...,Okay,2,Good,1


### Extract Root Words from Reviews

In [119]:
corpus = []
for i in range(0,1981):
#     try:
    review = re.sub("[^a-zA-z]", " ",
                   df_c["Reviews"][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)
#     except Exception as e:
#         pass

In [180]:
corpus[:3]

['burger lover today need work admiralti thu lunch pacif place find restaur avail lunch saw shop shackburg get doubl decker natur angu beef beef cook medium combin good potato chip crispi import point chees sauc amazingit special blend cheddar american chees raspberri lemonad copilot orang red special acid good',
 'famouli burger place hk quit crowd lunch hour line order food wait prepar food min whole order wait time find seat full lucki get seat help friendli staff gener din experi cool burger super awesom littl spice make excit tast bread soft oh wanna go back eat fri must tri item good much differ place nexttim would ask gravi sauc easi find poutin hk would definit recommenc restaur love come back love burger',
 'love ambianc comfort food cozi space got small children corner make place even better famili weekend holiday serv brunch menu banana pancak almond honey sea salt pancak moist tasti although bit burnt bottom part toast brioch ricotta blueberri preserv lemon serv cold realli

In [123]:
from sklearn.feature_extraction.text import CountVectorizer

# trasnform the vectorized review into array
cv = CountVectorizer(max_features = 150)
X = cv.fit_transform(corpus).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 1, 1, 0]])

In [154]:
len(corpus)

1981

In [164]:
len(df_c["Sentiment_Code"])

1981

### Model 1: Multinomial NB (0.79)

In [168]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(corpus).toarray()
y = df_c["Sentiment_Code"]

In [169]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   test_size = 0.2,
                                                   random_state = 0)

In [170]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
%matplotlib inline

mnb = MultinomialNB()
mnb.fit(X_train, y_train)
predmnb = mnb.predict(X_test)

print("Confusion Matrix for MNB:")
print(confusion_matrix(y_test,predmnb))
print("Score:", round(accuracy_score(y_test,predmnb)*100,2))
print("Classification Report:", classification_report(y_test,predmnb))

Confusion Matrix for MNB:
[[ 57  28]
 [ 53 259]]
Score: 79.6
Classification Report:               precision    recall  f1-score   support

           0       0.52      0.67      0.58        85
           1       0.90      0.83      0.86       312

    accuracy                           0.80       397
   macro avg       0.71      0.75      0.72       397
weighted avg       0.82      0.80      0.80       397



In [171]:
from sklearn import metrics

print("MNB Accuracy", metrics.accuracy_score(y_test, predmnb))

MNB Accuracy 0.7959697732997482


### Model 2: Random Forest (0.77)

In [172]:
from sklearn.ensemble import RandomForestClassifier

rmfr = RandomForestClassifier()
rmfr.fit(X_train, y_train)
predrmfr = rmfr.predict(X_test)
print("Confusion Matrix for Random Forest Classifier")
print(confusion_matrix(y_test,predrmfr))
print("Score", round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report", 
      classification_report(y_test, predrmfr))

Confusion Matrix for Random Forest Classifier
[[ 25  60]
 [ 32 280]]
Score 76.83
Classification Report               precision    recall  f1-score   support

           0       0.44      0.29      0.35        85
           1       0.82      0.90      0.86       312

    accuracy                           0.77       397
   macro avg       0.63      0.60      0.61       397
weighted avg       0.74      0.77      0.75       397



/Users/makhy/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [ ]:
print("Random Forest Accuracy", metrics.accuracy_score(y_test, predrmfr))

### Model 3: Gradient Boosting (0.83) ** Best!!

In [177]:
from sklearn.ensemble import GradientBoostingClassifier

gbe = GradientBoostingClassifier(random_state = 0)
parameters = {
    "learning_rate": [0.2, 0.4],
#     "max_features": [0.5,1],
    "max_depth": [3,5]
}
gridsearch = GridSearchCV(gbe, parameters,
                         cv = 5,
                         scoring = "roc_auc")
gridsearch.fit(X,y)

In [175]:
print(gridsearch.best_params_)
print(gridsearch.best_score_)

{'learning_rate': 0.2, 'max_depth': 3}
0.8419444905843219


In [178]:
# Boosting
gbi = GradientBoostingClassifier(learning_rate = 0.2,
                                max_depth = 3,
                                random_state = 0)

gbi.fit(X_train, y_train)
predgbi = gbi.predict(X_test)
print("Confusion Matrix for Gradient Boosting Classifier")
print(confusion_matrix(y_test,predgbi))
print("Score", round(accuracy_score(y_test,predgbi)*100,2))

Confusion Matrix for Gradient Boosting Classifier
[[ 33  52]
 [ 12 300]]
Score 83.88


In [179]:
print("Gradient Boosting Accuracy", metrics.accuracy_score(y_test, predgbi))

Gradient Boosting Accuracy 0.8387909319899244


### Prediction

In [232]:
corpus2 = []
rev = re.sub("[^a-zA-z]", " ",text)
rev = rev.lower()
rev = rev.split()
ps = PorterStemmer()
rev = [ps.stem(word) for word in rev if not word in set(stopwords.words("english"))]
rev = " ".join(rev)
corpus2.append(rev)

In [242]:
# Predict the sentiment of this comment
text = "The lobster is so delicious. The decor and atmosphere are fantastic.  It's overall a very pleasant expirience.  Will definitely come again!"

In [236]:
cv2 = CountVectorizer(max_features = 1000)
X2 = cv.fit_transform(corpus + corpus2).toarray()
my = X2[-1].reshape(1,-1)
result = gbi.predict(my)

In [240]:
if result == 1:
    print("Good")
else:
    print("Bad")

Good
